# Set the paths 

In [ ]:
import os


projectPath = os.path.abspath("..")
dat_path = os.path.join(projectPath, "dat")
res_path = os.path.join(projectPath, "res")
rearm_path = os.path.join(dat_path, "ReArm.lnk")
#res_info_path = os.path.join(res_path, "CheckFilesInRearmVisit")


In [ ]:
# set the flag to NOT run the tests
doRunTests = False

# run the outside notebooks (to get the functions)
%run -i "checkFilesInVisit.ipynb"
%run -i "kinect-to-mouse-time-correction.ipynb"


# set the visits to check
visits = [
    "ReArm_C1P02",
    "ReArm_C1P023",
]

# check the files in the visit
for visit in visits:
    print("###############################################")
    print("Checking visit: ", visit)

    visit_path = os.path.join(rearm_path, visit)
    if not os.path.isdir(visit_path):
        print("ERROR: pathToData is not a directory: ", visit_path)
    else:
        visitToCheck = CheckFilesInRearmVisit(visit_path)
        kinectToMouseTimeCorrection(visit_path)